In [ ]:
import os
import shutil
import numpy as np

# 设置路径
original_dataset_dir = './DATA/train'
base_dir = './DATA'
train_dir = os.path.join(base_dir, 'train1')
validation_dir = os.path.join(base_dir, 'validation1')

# 创建目录
if not os.path.exists(base_dir):
    os.mkdir(base_dir)
if not os.path.exists(train_dir):
    os.mkdir(train_dir)
if not os.path.exists(validation_dir):
    os.mkdir(validation_dir)

# 创建训练和验证集的cats和dogs目录
train_cats_dir = os.path.join(train_dir, 'cats')
os.mkdir(train_cats_dir)
train_dogs_dir = os.path.join(train_dir, 'dogs')
os.mkdir(train_dogs_dir)
validation_cats_dir = os.path.join(validation_dir, 'cats')
os.mkdir(validation_cats_dir)
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
os.mkdir(validation_dogs_dir)

# 获取所有图片文件名
cat_fnames = [f for f in os.listdir(os.path.join(original_dataset_dir, 'cats')) if f.endswith('.jpg')]
dog_fnames = [f for f in os.listdir(os.path.join(original_dataset_dir, 'dogs')) if f.endswith('.jpg')]

# 定义分割比例
train_size = 0.8
validation_size = 0.2

# 分割训练和验证集
np.random.seed(42)
np.random.shuffle(cat_fnames)
np.random.shuffle(dog_fnames)

train_cat_fnames = cat_fnames[:int(len(cat_fnames) * train_size)]
validation_cat_fnames = cat_fnames[int(len(cat_fnames) * train_size):]

train_dog_fnames = dog_fnames[:int(len(dog_fnames) * train_size)]
validation_dog_fnames = dog_fnames[int(len(dog_fnames) * train_size):]

# 复制文件
for fname in train_cat_fnames:
    src = os.path.join(original_dataset_dir, 'cats', fname)
    dst = os.path.join(train_cats_dir, fname)
    shutil.copyfile(src, dst)

for fname in validation_cat_fnames:
    src = os.path.join(original_dataset_dir, 'cats', fname)
    dst = os.path.join(validation_cats_dir, fname)
    shutil.copyfile(src, dst)

for fname in train_dog_fnames:
    src = os.path.join(original_dataset_dir, 'dogs', fname)
    dst = os.path.join(train_dogs_dir, fname)
    shutil.copyfile(src, dst)

for fname in validation_dog_fnames:
    src = os.path.join(original_dataset_dir, 'dogs', fname)
    dst = os.path.join(validation_dogs_dir, fname)
    shutil.copyfile(src, dst)

print('训练集和验证集划分完成。')

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import datetime

# 设置路径
train_dir = './DATA/train1'
validation_dir = './DATA/validation1'

# 定义训练和验证数据生成器
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='sparse'  # 使用稀疏类别模式
)

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='sparse'  # 使用稀疏类别模式
)

# 使用VGG19模型
conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# 冻结卷积基
conv_base.trainable = False

# 定义模型
model = Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))  # 改为2个神经元，使用softmax激活函数

# 编译模型
model.compile(optimizer=Adam(lr=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 设置TensorBoard日志
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# 定义EarlyStopping回调
early_stopping_callback = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)

# 训练模型
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 20,
    epochs=100,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // 20,
    callbacks=[tensorboard_callback, early_stopping_callback]
)

# 保存模型
# model.save('cats_and_dogs_classifier.h5')


Found 4000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


Epoch 1/100
200/200 [==============================] - 31s 153ms/step - loss: 0.9355 - accuracy: 0.7287 - val_loss: 0.2921 - val_accuracy: 0.8820
Epoch 2/100
200/200 [==============================] - 97s 486ms/step - loss: 0.5171 - accuracy: 0.7880 - val_loss: 0.2994 - val_accuracy: 0.8830
Epoch 3/100
200/200 [==============================] - 106s 531ms/step - loss: 0.5383 - accuracy: 0.7872 - val_loss: 0.3579 - val_accuracy: 0.8560
Epoch 4/100
200/200 [==============================] - ETA: 0s - loss: 0.4778 - accuracy: 0.8052

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('cats_and_dogs_classifier.h5')

In [ ]:
# 预测函数
def load_and_preprocess_image(img_path, target_size=(150, 150)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0
    return img_array

def predict_image(model, img_array):
    prediction = model.predict(img_array)
    class_idx = np.argmax(prediction[0])
    class_labels = ['cat', 'dog']
    return class_labels[class_idx]

# img_path = './DATA/train/dogs/dog.7.jpg'
img_path = './DATA/train/cats/cat.27.jpg'
img_array = load_and_preprocess_image(img_path)
result = predict_image(model, img_array)
print(f'The image is predicted to be a {result}.')

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
model.summary()